WARNING: This notebook has some dependency issues !!!

In [ ]:
# !pip install -q accelarate

In [ ]:
# !pip install -q peft transformers bitsandbytes trl

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel

In [ ]:
!pip install -q datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [ ]:
from datasets import load_dataset
import re

dataset = load_dataset('timdettmers/openassistant-guanaco')

dataset = dataset['train'].shuffle(seed=42).select(range(1000))

def transform_conversation(example):
  conversation_text = example['text']
  segments = conversation_text.split('###')
  reformatted_segments = []

  for i in range(1, len(segments) - 1, 2):
    human_text = segments[i].strip().replace('Human:','').strip()

    if i+1<len(segments):
      assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()
      reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
    else:
      reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

  return {'text': ''.join(reformatted_segments)}

transformed_dataset = dataset.map(transform_conversation)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
transformed_dataset.push_to_hub("guanaco-llama2-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/PranavBalajiRS/guanaco-llama2-1k/commit/a6671a5560ec526dbfd1378bca79fd67eb042835', commit_message='Upload dataset', commit_description='', oid='a6671a5560ec526dbfd1378bca79fd67eb042835', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/PranavBalajiRS/guanaco-llama2-1k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='PranavBalajiRS/guanaco-llama2-1k'), pr_revision=None, pr_num=None)

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

In [ ]:
# QLORA PARAMETERS

lora_r = 64 #LoRA attention dimension
lora_alpha = 16 #Alpha Parameters for LoRA scaling
lora_dropout = 0.1 #Dropout probability for LoRA layers

In [ ]:
# BITSANDBYTES PARAMETERS

use_4bit = True #Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16" #Compute dtype for 4 bit base models
bnb_4bit_quant_type = "nf4" #Quantization type (fp4 or nf4)
use_nested_quant = False #Activate nested quantization for 4-bit base models (double quantization)

In [ ]:
# TRAINING ARGUMENTS PARAMETERS

output_dir = "./results"
num_train_epochs = 1

#Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = True

per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "pages_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

In [ ]:
# SFT PARAMETERS

max_seq_length = None
packing = False
device_map = {"": 0}

In [ ]:
dataset = load_dataset(dataset_name, split="train")

# Loading tokenizer and model with  QLORA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = bnb_4bit_quant_type,
    bnb_4bit_compute_dtype = compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant,
)

In [ ]:
# Check GPU compatability with bfloat16
if compute_dtype == torch.float16 and use_4bit:
  major, _ = torch.cuda.get_device_capability()
  if major >= 8:
    print("=" * 80)
    print("Your GPU supports bfloat16: accelerate training with bf16=True")
    print("=" * 80)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = device_map
)

model.config.use_cache = True
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix overflow issue with fp16 training

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    bias = "none",
    task_type = "CAUSAL_LM"
)

In [ ]:
# Set Training parameters
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = "paged_adamw_32bit",
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = learning_rate,
    weight_decay = weight_decay,
    fp16 = True,
    bf16 = False,
    max_grad_norm = max_grad_norm,
    max_steps = max_steps,
    warmup_ratio = warmup_ratio,
    group_by_length = group_by_length,
    lr_scheduler_type = lr_scheduler_type,
    report_to = "tensorboard"
)

In [ ]:
# pip install -U trl

In [ ]:
import trl
print(trl.__version__)

0.20.0.dev0


In [ ]:
from trl import SFTTrainer
print(SFTTrainer.__init__.__code__.co_varnames)

('self', 'model', 'args', 'data_collator', 'train_dataset', 'eval_dataset', 'processing_class', 'compute_loss_func', 'compute_metrics', 'callbacks', 'optimizers', 'optimizer_cls_and_kwargs', 'preprocess_logits_for_metrics', 'peft_config', 'formatting_func', 'model_id', 'model_name', 'dict_args', 'eos_token', 'eos_token_id', 'chat_template_file', 'first_example', 'pad_token', 'pad_token_id', 'preprocess_dataset')


In [ ]:
# !pip uninstall -y trl
# !pip uninstall -y transformers peft
# !pip install transformers==4.38.2 peft==0.7.1 bitsandbytes==0.41.3 trl==0.7.4

In [ ]:
# !pip uninstall -y trl
# !pip install git+https://github.com/huggingface/trl.git

In [ ]:
def formatting_func(example):
    return example["text"]

def preprocess(example):
    text = formatting_func(example)
    tokenized = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=512,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess, remove_columns=dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_arguments,
    peft_config=peft_config,
)


Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST]{prompt}[/INST]")
print(result[0]['generated_text'])

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# SAVING THE FINETUNED MODEL

# Reload  model in the FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map = device_map
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

#reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# PUSH MODEL TO HUGGING FACE HUB

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!huggingface-cli login
model.push_to_hub("PranavBalajiRS/Llama-2-7b-chat-finetune", check_pr = True)
tokenizer.push_to_hub("PranavBalajiRS/Llama-2-7b-chat-finetune", check_pr = True)